<a href="https://colab.research.google.com/github/zhh25/Titanic/blob/main/Fine_Tune_Random_Forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import os
path_root = '/content/drive/MyDrive/titanic'
train = pd.read_csv(os.path.join(path_root, 'data/train.csv'))
test =  pd.read_csv(os.path.join(path_root, 'data/test.csv'))
sample_submission =  pd.read_csv(os.path.join(path_root, 'data/sample_submission.csv'))
titanic = train.drop('Survived', axis = 1)
titanic_label = train['Survived'].copy()

# Transformation Pipeline




In [ ]:
exec(open('/content/drive/MyDrive/Colab Notebooks/titanic_data_pipeline.py' ).read())


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
titanic_prepared = full_pipeline.fit_transform(titanic)
test_prepared = full_pipeline.transform(test)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(titanic_prepared, titanic_label, test_size = 0.25,  )

In [ ]:
cols_full = attribs_names()

# RandomizedSearchCV

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
random_grid = {'n_estimators': [int(x) for x in np.linspace(50, 500, num = 10)],
               'max_depth': [int(x) for x in np.linspace(5, 20 , num = 5)],
               'min_samples_split': [2,5,10, 15, 20],
               'min_samples_leaf': [1,2,4]
               }

In [ ]:
from pprint import pprint
pprint(random_grid)

{'max_depth': [5, 8, 12, 16, 20],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10, 15, 20],
 'n_estimators': [50, 100, 150, 200, 250, 300, 350, 400, 450, 500]}


In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rf_tuner = RandomizedSearchCV(estimator = RandomForestClassifier(),
                              param_distributions = random_grid,
                              n_iter=50,
                              cv = 4,
                              verbose = 1,
                              random_state = 25,
                              n_jobs = -1
                              )

In [ ]:
rf_tuner.fit(titanic_prepared, titanic_label)

Fitting 4 folds for each of 50 candidates, totalling 200 fits


RandomizedSearchCV(cv=4, estimator=RandomForestClassifier(), n_iter=50,
                   n_jobs=-1,
                   param_distributions={'max_depth': [5, 8, 12, 16, 20],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10, 15, 20],
                                        'n_estimators': [50, 100, 150, 200, 250,
                                                         300, 350, 400, 450,
                                                         500]},
                   random_state=25, verbose=1)

In [ ]:
rf_tuner.best_params_

{'max_depth': 16,
 'min_samples_leaf': 4,
 'min_samples_split': 10,
 'n_estimators': 400}

In [ ]:
rf_tuner.best_score_

0.8395043025087867

In [ ]:
best_model = rf_tuner.best_estimator_

In [ ]:
sorted(zip(best_model.feature_importances_, cols_full), reverse = True)

[(0.18373455240566713, 'sex'),
 (0.17869685190852802, 'name_Mr'),
 (0.0993909751819813, 'fare'),
 (0.0750189557392328, 'Pclass'),
 (0.07197110141391912, 'age'),
 (0.062016499421002305, 'name_Miss'),
 (0.05204222100125977, 'name_Mrs'),
 (0.04405802000007865, 'ticket'),
 (0.043116177892985795, 'cabin_M'),
 (0.03502802644813525, 'family_recode'),
 (0.030173917324005175, 'family'),
 (0.01685953437629001, 'SibSp_recode'),
 (0.016665299618217215, 'SibSp'),
 (0.014292376646287885, 'cabin_DE'),
 (0.012384749107535885, 'child'),
 (0.010910072938241838, 'embarled_S'),
 (0.010194772056646697, 'name_Master'),
 (0.009535899876326784, 'Parch'),
 (0.008412957024209468, 'cabin_ABC'),
 (0.007706622140632968, 'embarled_C'),
 (0.0066688694657283665, 'Parch_recode'),
 (0.005982221930233715, 'name_Officer'),
 (0.004555833618918662, 'embarled_Q'),
 (0.0004241826414538924, 'cabin_FG'),
 (0.00011926601188817236, 'senior'),
 (4.0043810593299354e-05, 'name_Royalty')]

## Predict

In [ ]:
test_prob = best_model.predict(test_prepared)

In [ ]:
np.sum(test_prob )


153

In [ ]:
sample_submission['Survived'] = (test_prob)
#sample_submission['Survived'] = sample_submission['Survived'].astype('int8')
sample_submission

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [ ]:
sample_submission.to_csv(os.path.join(path_root, 'submission/RandomForest_1.csv'), index = False)

# Sklearn Tuner

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics
! pip install keras_tuner
import keras_tuner as kt

In [ ]:
class RF_HyperModel(kt.HyperModel):
    def build(self, hp):
        model = RandomForestClassifier(
            n_estimators = hp.Int('n_estimators', 50, 1000, step = 10),
            #max_features = hp.Choice('max_features', ['sqrt', 'log2'])
            max_depth = hp.Int('max_depth', 5, 30, step = 2),
            min_samples_split= hp.Int('min_sample_split', 2, 30, 4),
            min_samples_leaf= hp.Int('min_samples_leaf', 1,5,2),
            oob_score = False,
            n_jobs = -1
        )
        return model     

In [ ]:
tuner = kt.SklearnTuner(
    hypermodel = RF_HyperModel(),
    oracle = kt.oracles.BayesianOptimizationOracle(
        objective = kt.Objective('score', 'max'),
        max_trials = 100,
        #executions_per_trial=2
    ),
    scoring = metrics.make_scorer(metrics.accuracy_score),
    cv = StratifiedKFold(4),
    directory = os.path.join(path_root, 'tuner/random_forest'),
    project_name = 'v1',
    overwrite = True,
    
)



In [ ]:
tuner.search(titanic_prepared, np.array(titanic_label))

In [ ]:
tuner.results_summary()

Results summary
Results in /content/drive/MyDrive/titanic/tuner/random_forest/v1
Showing 10 best trials
Trial summary
Hyperparameters:
n_estimators: 270
max_depth: 19
min_sample_split: 14
min_samples_leaf: 3
Score: 0.8417414050822123
Trial summary
Hyperparameters:
n_estimators: 400
max_depth: 19
min_sample_split: 14
min_samples_leaf: 3
Score: 0.8406354785278551
Trial summary
Hyperparameters:
n_estimators: 430
max_depth: 21
min_sample_split: 10
min_samples_leaf: 3
Score: 0.8395144022946714
Trial summary
Hyperparameters:
n_estimators: 1000
max_depth: 29
min_sample_split: 2
min_samples_leaf: 5
Score: 0.8395144022946714
Trial summary
Hyperparameters:
n_estimators: 490
max_depth: 19
min_sample_split: 14
min_samples_leaf: 3
Score: 0.8395093524017292
Trial summary
Hyperparameters:
n_estimators: 340
max_depth: 17
min_sample_split: 10
min_samples_leaf: 3
Score: 0.8395093524017291
Trial summary
Hyperparameters:
n_estimators: 430
max_depth: 21
min_sample_split: 10
min_samples_leaf: 3
Score: 0.839

## Retrain the model


Find the optimal number of epochs to train the model with the hyperparameters obtained from the search.

In [ ]:
best_hps = tuner.get_best_hyperparameters(1)[0]
print(best_hps.get('n_estimators'), 
      best_hps.get('max_depth'), 
      best_hps.get('min_sample_split'), 
      best_hps.get('min_samples_leaf'))

270 19 14 3


In [ ]:
model = RandomForestClassifier(
            n_estimators = best_hps.get('n_estimators'),
            max_depth = best_hps.get('max_depth'),
            min_samples_split= best_hps.get('min_sample_split'),
            min_samples_leaf= best_hps.get('min_samples_leaf'),
            oob_score = True,
            n_jobs = -1
        )
model.fit(titanic_prepared, titanic_label)


RandomForestClassifier(max_depth=19, min_samples_leaf=3, min_samples_split=14,
                       n_estimators=270, n_jobs=-1, oob_score=True)

In [ ]:
model.oob_score_

0.8372615039281706

In [ ]:
sorted(zip(model.feature_importances_, cols_full), reverse = True)

[(0.1904101266256227, 'name_Mr'),
 (0.18431371671356325, 'sex'),
 (0.09904708079334662, 'fare'),
 (0.07228351896411482, 'age'),
 (0.06977385204820145, 'Pclass'),
 (0.05609077680269243, 'name_Miss'),
 (0.052701588769020584, 'name_Mrs'),
 (0.04155528437494137, 'ticket'),
 (0.037466228917410166, 'cabin_M'),
 (0.036823172391362013, 'family_recode'),
 (0.030230867914952813, 'family'),
 (0.018046641811460845, 'SibSp'),
 (0.017426186506878494, 'cabin_DE'),
 (0.015747883379001727, 'SibSp_recode'),
 (0.011893492476911734, 'child'),
 (0.010990569688375777, 'name_Master'),
 (0.010604998098862077, 'embarled_S'),
 (0.00905868750884864, 'Parch'),
 (0.008775442530179185, 'name_Officer'),
 (0.007883023225705907, 'cabin_ABC'),
 (0.007200385833429042, 'embarled_C'),
 (0.005661763051920646, 'Parch_recode'),
 (0.004706860401397896, 'embarled_Q'),
 (0.0006376332272123714, 'cabin_FG'),
 (0.0004897803443926235, 'senior'),
 (0.0001804376001948162, 'name_Royalty')]

## predict

In [ ]:
test_prob = model.predict(test_prepared)

In [ ]:
np.sum(test_prob )


157

In [ ]:
sample_submission['Survived'] = (test_prob)
#sample_submission['Survived'] = sample_submission['Survived'].astype('int8')
sample_submission

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [ ]:
sample_submission.to_csv(os.path.join(path_root, 'submission/RandomForest_2_kerastuner.csv'), index = False)

In [ ]:
model

RandomForestClassifier(max_depth=19, min_samples_leaf=3, min_samples_split=14,
                       n_estimators=270, n_jobs=-1, oob_score=True)

# Hyperop

In [ ]:
from hyperopt import fmin, tpe, hp, anneal, Trials

In [ ]:
space = {'n_estimators': hp.quniform('n_estimators', 50, 1000, 10),    
        'max_depth': hp.quniform('max_depth', 5, 30, 2),
        'min_samples_split':  hp.quniform('min_sample_split', 2, 30, 4),
        'min_samples_leaf':  hp.quniform('min_samples_leaf', 1,5,2),
        'oob_score': True}

In [ ]:
from sklearn.ensemble import RandomForestClassifier
def objective(space):
    errors = []
    for i in range(4):
        model = RandomForestClassifier(
                    n_estimators = int(space['n_estimators']),
                    max_depth = int(space['max_depth']),
                    min_samples_split= int(space['min_samples_split']),
                    min_samples_leaf= int(space['min_samples_leaf']),
                    oob_score = space['oob_score']
                ) 
        model.fit(titanic_prepared, titanic_label)
        errors.append( - model.oob_score_)
    return  np.mean(errors)

In [ ]:
trials = Trials()

best_hps = fmin(fn = objective,
                space = space,
                algo = tpe.suggest,
                max_evals = 200,
                trials = trials)

100%|██████████| 200/200 [16:46<00:00,  5.03s/it, best loss: -0.8395061728395061]


In [ ]:
errors = trials.losses()
accuracies = pd.DataFrame([- error for error in errors], columns = ['acc'])
best_acc = accuracies.sort_values(by = 'acc', ascending=False)[:10]
best_ixs = list(best_acc.index)
for ix in best_ixs:
    trial = trials.trials[ix]
    print(ix)
    print( - trial['result']['loss'])
    print(trial['misc']['vals'])
    #print(trial['result']['best_iters'],
          #np.mean(trial['result']['best_iters']))

90
0.8395061728395061
{'max_depth': [14.0], 'min_sample_split': [16.0], 'min_samples_leaf': [4.0], 'n_estimators': [730.0]}
193
0.8395061728395061
{'max_depth': [8.0], 'min_sample_split': [12.0], 'min_samples_leaf': [4.0], 'n_estimators': [1000.0]}
55
0.8392255892255892
{'max_depth': [22.0], 'min_sample_split': [16.0], 'min_samples_leaf': [4.0], 'n_estimators': [930.0]}
179
0.8389450056116723
{'max_depth': [10.0], 'min_sample_split': [16.0], 'min_samples_leaf': [4.0], 'n_estimators': [510.0]}
137
0.8386644219977553
{'max_depth': [10.0], 'min_sample_split': [12.0], 'min_samples_leaf': [4.0], 'n_estimators': [810.0]}
98
0.8383838383838385
{'max_depth': [12.0], 'min_sample_split': [12.0], 'min_samples_leaf': [4.0], 'n_estimators': [870.0]}
131
0.8383838383838385
{'max_depth': [22.0], 'min_sample_split': [20.0], 'min_samples_leaf': [4.0], 'n_estimators': [340.0]}
34
0.8383838383838385
{'max_depth': [14.0], 'min_sample_split': [12.0], 'min_samples_leaf': [4.0], 'n_estimators': [420.0]}
109


## retrain

In [ ]:
best_model = RandomForestClassifier(
    n_estimators=int(best_hps['n_estimators']),
    min_samples_split=int(best_hps['min_sample_split']),
    min_samples_leaf= int(best_hps['min_samples_leaf']),
    max_depth=int(best_hps['max_depth']),
    oob_score=True)
best_model.fit(titanic_prepared, titanic_label)
best_model.oob_score_

0.8395061728395061

## predict

In [ ]:
best_model.predict(test_prepared)
sample_submission['Survived'] = best_model.predict(test_prepared)
#sample_submission['Survived'] = sample_submission['Survived'].astype('int8')
sample_submission

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [ ]:
sample_submission.to_csv(os.path.join(path_root, 'submission/RandomForest_hyperop.csv'), index = False)

## save 

In [ ]:
import joblib

In [ ]:
joblib.dump(best_model, os.path.join(path_root, 'models/best_rf.sav'))

['/content/drive/MyDrive/titanic/models/best_rf.sav']

In [ ]:
best_rf = joblib.load(os.path.join(path_root, 'models/best_rf.sav'))

In [ ]:
best_rf.predict(test_prepared)

array([0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0,
       1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1,
       0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0,